In [4]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify

def create_patch(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of image filenames in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        # Read the image
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Unable to open image file {image_path}")
            continue

        # Making patches
        patches_img = patchify(image, (256, 256, 3), step = 256)
        # print(patches_img.shape)
        for i in range(patches_img.shape[0]): 
            for j in range(patches_img.shape[1]): 
                single_patch_img = patches_img[i, j, :, :]
                single_patch_img = single_patch_img[0]
                mask_filename = os.path.join(output_folder, f"{image_file[:-4]}_{i}_{j}.jpg")
                cv2.imwrite(mask_filename, single_patch_img)

if __name__ == "__main__":
    # Input folder containing annotated images
    input_folder = "data/p_val_masks"
    
    # Output folder for masks
    output_folder = "data/val_masks"

    # Create masks
    create_mask(input_folder, output_folder)



Masks created and saved.


In [13]:
import os
import random
import cv2

# Define the paths to the folders containing the images
mask_folder = "data/train_masks"
img_folder = "data/train_images"

# Iterate through the images in the mask folder
for mask_file in os.listdir(mask_folder):
    if mask_file.startswith("mask_img_") and mask_file.endswith(".jpg"):
        mask_path = os.path.join(mask_folder, mask_file)
        img_file = mask_file.replace("mask_img_", "img_")
        img_path = os.path.join(img_folder, img_file)

        # Check if the mask image is all black
        mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if cv2.countNonZero(mask_image) == 0:
            # Image is all black
            # print(f"{mask_file} is all black. Deleting with 90% chance.")
            # Decide whether to delete the corresponding image with 90% chance
            if random.random() < 0.8:
                os.remove(mask_path)
                if os.path.exists(img_path):
                    os.remove(img_path)
        #             print(f"{img_file} deleted.")
        #     else:
        #         print(f"{img_file} not deleted (10% chance).")
        # else:
        #     print(f"{mask_file} is not all black.")
